In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

In [ ]:
import ee
import geemap
ee.Initialize(project="thurgau-irrigation")
# ee.Initialize(project="ee-sahellakes")

from src.vegetation_period_NDVI.time_series import extract_time_ranges, get_harmonic_ts
from src.vegetation_period_NDVI.vegetation_period_extraction import (
    get_crop_veg_period,
    create_binary_ndvi_indicator,
    create_binary_mask,
)
from typing import List

In [ ]:
ee.Initialize(project="thurgau-irrigation")
# ee.Initialize(project="ee-sahellakes")

## Exporting the vegetation period images as an asset to the project
### Define the Area of Interest

In [ ]:
adm1_units = ee.FeatureCollection('projects/thurgau-irrigation/assets/GIS/Kantone_simplified100m')

#filter the cantons Zürich, Schaffhausen, and Thurgau from adm1_units using the 'NAME' property
adm1_units = adm1_units.filter(ee.Filter.inList('NAME', ['Zürich', 'Schaffhausen', 'Thurgau']))
aoi = adm1_units.geometry()
jurisdictions = adm1_units

### Define the years of interest

In [ ]:
years = [2018]  # 2018,2024, 2023, 2022, 2021, 2020, 2019


# Constants
NDVI_THRESHOLD = 0.35
NDVI_LOW_THRESHOLD = 0.3
AGG_INTERVAL = 15


### Export the Vegetation Periods images to the project assets

In [ ]:
for year in years:

    print(f"Processing year {year}")

    time_intervals = extract_time_ranges([f"{year}-04-01", f"{year}-10-31"], AGG_INTERVAL)

    image_to_export = get_crop_veg_period(year, aoi, time_intervals,NDVI_THRESHOLD,NDVI_LOW_THRESHOLD)

    task_name = f"ZH_SH_TG_crop_veg_period_{year}"
    asset_id = f"projects/thurgau-irrigation/assets/ZH_SH_TG/crop_vegetation_period_{year}_harmonic"

    bands_to_export = [
        "firstStart",
        "firstEnd",
        "secondStart",
        "secondEnd",
        "isDoubleCropping",
    ]

    image_to_export = image_to_export.select(bands_to_export).clip(aoi)

    task = ee.batch.Export.image.toAsset(
        image=image_to_export,
        description=task_name,
        assetId=asset_id,
        region=aoi,
        scale=10,
        maxPixels=1e13,
    )

    task.start()

    print(f"Exporting {task_name} to {asset_id}")

### Add Vegetation Period info and Crop Class to Landuse collection

In [ ]:
# Import the necessary functions
from src.et_green.filter_nutzungsflaechen import get_winter_crops
from src.et_blue_per_field.field_level_postprocessing import FieldLevelETProcessor,ProcessingConfig, CropTypeConfig, add_double_cropping_info, add_crop_class_to_fields


MINIMUM_FIELD_SIZE = 1000  # minimum field size in m^2

for year in years:

    print(f"Processing year {year}")

    # Create processing configuration
    config = ProcessingConfig(
        year=year,
        landuse_path="projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_{year}_Kulturen",
        vegetation_period_path="projects/thurgau-irrigation/assets/ZH_SH_TG/crop_vegetation_period_{year}_harmonic",
        landuse_property_name="nutzung"
    )
    processor = FieldLevelETProcessor(config)

    # Use the globally configured canton for field processing
    # Filter for crop types already defined in FieldLevelETProcessor plus winter crops
    # (winter crops are important as there might be second crops grown there which are irrigated)


    # Load the landuse collection for the configured canton
    landuse_path = config.landuse_path.format(year=year)
    print(f"Loading landuse data from: {landuse_path}")
    landuse_collection = ee.FeatureCollection(landuse_path)
    # Make unique ID
    landuse_collection = landuse_collection.map(lambda f: f.set('field_id', f.id()))
    print(f"Landuse collection: {landuse_path}")

    # Get all crop types defined in FieldLevelETProcessor
    processor_crop_types = set()
    for crop_config in processor.crop_configs.values():
        processor_crop_types.update(crop_config.landuse_categories)

    # Get winter crops
    winter_crops = get_winter_crops()

    # Combine all crop types we want to include
    target_crop_types = processor_crop_types.union(winter_crops)

    print(f"Crop types from FieldLevelETProcessor: {len(processor_crop_types)}")
    print(f"Winter crop types: {len(winter_crops)}")
    print(f"Total target crop types: {len(target_crop_types)}")
    print(f"Sample target crops: {list(target_crop_types)[:10]}")

    # Create filter for target crop types
    crop_filter = ee.Filter.inList(config.landuse_property_name, list(target_crop_types))

    # Filter the landuse collection for target crops and minimum field size
    filtered_fields = landuse_collection.filter(crop_filter).filter(
        ee.Filter.gte('flaeche_m2', MINIMUM_FIELD_SIZE)
    )

    # Add crop class to fields
    filtered_fields = add_crop_class_to_fields(filtered_fields)

    vegetation_period = ee.Image(
        config.vegetation_period_path.format(year=config.year)
    )
    print('veg period image:', vegetation_period.bandNames().getInfo())

    filtered_fields=add_double_cropping_info(filtered_fields,vegetation_period)

    #Export the fields to asset

    print('number of fields', filtered_fields.size().getInfo())
    task = ee.batch.Export.table.toAsset(
        collection=filtered_fields,
        description=f"ZH_SH_TG_fields_with_veg_period_{year}",
        assetId=f"projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_{year}_Kulturen_with_veg_period_and_crop_class",
    )

    task.start()

    print(f"Exporting fields of {year} to asset")

### Process Landuse assets where Crop Class still missing

In [ ]:
# Import the necessary functions
from src.et_green.filter_nutzungsflaechen import get_winter_crops
from src.et_blue_per_field.field_level_postprocessing import FieldLevelETProcessor,ProcessingConfig, CropTypeConfig, add_double_cropping_info, add_crop_class_to_fields

years = [2018]


MINIMUM_FIELD_SIZE = 1000  # minimum field size in m^2

for year in years:

    print(f"Processing year {year}")

    # Create processing configuration
    config = ProcessingConfig(
        year=year,
        landuse_path="projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_{year}_Kulturen_with_veg_period",
        # vegetation_period_path="projects/thurgau-irrigation/assets/ZH_SH_TG/crop_vegetation_period_{year}_harmonic",
        landuse_property_name="nutzung"
    )
    processor = FieldLevelETProcessor(config)

    # Use the globally configured canton for field processing
    # Filter for crop types already defined in FieldLevelETProcessor plus winter crops
    # (winter crops are important as there might be second crops grown there which are irrigated)


    # Load the landuse collection for the configured canton
    landuse_path = config.landuse_path.format(year=year)
    print(f"Loading landuse data from: {landuse_path}")
    landuse_collection = ee.FeatureCollection(landuse_path)
    # Make unique ID
    landuse_collection = landuse_collection.map(lambda f: f.set('field_id', f.id()))
    print(f"Landuse collection: {landuse_path}")

    # Get all crop types defined in FieldLevelETProcessor
    processor_crop_types = set()
    for crop_config in processor.crop_configs.values():
        processor_crop_types.update(crop_config.landuse_categories)

    # Get winter crops
    winter_crops = get_winter_crops()

    # Combine all crop types we want to include
    target_crop_types = processor_crop_types.union(winter_crops)

    print(f"Crop types from FieldLevelETProcessor: {len(processor_crop_types)}")
    print(f"Winter crop types: {len(winter_crops)}")
    print(f"Total target crop types: {len(target_crop_types)}")
    print(f"Sample target crops: {list(target_crop_types)[:10]}")

    # Create filter for target crop types
    crop_filter = ee.Filter.inList(config.landuse_property_name, list(target_crop_types))

    # Filter the landuse collection for target crops and minimum field size
    filtered_fields = landuse_collection.filter(crop_filter).filter(
        ee.Filter.gte('flaeche_m2', MINIMUM_FIELD_SIZE)
    )

    # Add crop class to fields
    filtered_fields = add_crop_class_to_fields(filtered_fields)

    # vegetation_period = ee.Image(
    #     config.vegetation_period_path.format(year=config.year)
    # )
    # print('veg period image:', vegetation_period.bandNames().getInfo())

    # filtered_fields=add_double_cropping_info(filtered_fields,vegetation_period)

    #Export the fields to asset

    print('number of fields', filtered_fields.size().getInfo())
    task = ee.batch.Export.table.toAsset(
        collection=filtered_fields,
        description=f"ZH_SH_TG_fields_with_veg_period_{year}",
        assetId=f"projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_{year}_Kulturen_with_veg_period_and_crop_class",
    )

    task.start()

    print(f"Exporting fields of {year} to asset")